# Laydown Planner - Data Analysis
Deep dive analysis of construction site laydown planning data.

## 1. Setup and Imports

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.data_loader import DataLoader

print('All imports successful!')

## 2. Load Data

In [ ]:
objects_df = pd.read_csv('../data/sample_objects.csv')
schedule_df = pd.read_csv('../data/sample_schedule.csv')

print(f'Objects: {len(objects_df)} rows')
print(f'Schedule: {len(schedule_df)} rows')

## 3. Objects Analysis

In [ ]:
print('=== Objects Overview ===')
print(f'Total Objects: {len(objects_df)}')
print(f'\nColumns: {objects_df.columns.tolist()}')
print(objects_df.head())

## 4. DXF Visualization

In [ ]:
import subprocess
try:
    import ezdxf
    print('ezdxf already installed')
except ImportError:
    print('Installing ezdxf...')
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'ezdxf'])
    import ezdxf

In [ ]:
import ezdxf
from ezdxf.render import matplotlib as mpl_renderer

dxf_file = '../data/sample_laydown.dxf'

try:
    doc = ezdxf.readfile(dxf_file)
    msp = doc.modelspace()
    
    print(f'✓ DXF loaded: {doc.dxfversion}')
    print(f'Entities: {len(msp)}')
    
    fig, ax = plt.subplots(figsize=(14, 10))
    mpl_renderer.draw_layout(ax, msp)
    ax.set_aspect('equal')
    ax.grid(True, alpha=0.3)
    ax.set_title('Laydown Plan Visualization')
    plt.show()
except Exception as e:
    print(f'Error: {e}')

## 5. Summary

In [ ]:
print('Analysis complete!')